In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Read the CSV and Perform Basic Data Cleaning

In [31]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [32]:
df.shape

(6991, 41)

# Select your features (columns)

In [33]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

My selection:
koi_disposition
koi_period
koi_time0bk
koi_impact
koi_duration
koi_depth
koi_prad
koi_teq
koi_insol
koi_model_snr
koi_steff
koi_slogg
koi_srad

In [34]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_steff', 'koi_slogg', 'koi_srad']]
selected_features.head()

,koi_disposition,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad
0,CONFIRMED,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,5455,4.467,0.927
1,FALSE POSITIVE,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,5853,4.544,0.868
2,FALSE POSITIVE,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,5805,4.564,0.791
3,CONFIRMED,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,6031,4.438,1.046
4,CONFIRMED,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,6046,4.486,0.972


# Create a Train Test Split

Use `koi_disposition` for the y values

In [35]:
y = selected_features["koi_disposition"]
print(y.shape)

(6991,)


In [36]:
y.head()

0         CONFIRMED
1    FALSE POSITIVE
2    FALSE POSITIVE
3         CONFIRMED
4         CONFIRMED
Name: koi_disposition, dtype: object

In [37]:
X = selected_features.drop("koi_disposition", axis=1)
print(X.shape)

(6991, 12)


In [38]:
#Split our data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [39]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad
3563,10.548413,139.064020,1.0170,1.8720,102.9,3.89,899,154.84,11.7,6047,4.387,1.092
4099,24.754385,140.207320,0.7090,3.3900,593.3,2.10,491,13.70,18.0,4852,4.519,0.804
5460,1.057336,131.792007,0.2620,1.5795,47337.0,14.59,1276,623.51,476.0,4664,4.594,0.683
1091,201.118319,187.569860,0.0010,10.3280,584.8,2.28,300,1.92,34.7,5646,4.447,0.954
5999,91.649983,175.715600,0.2136,10.2940,193.6,2.27,568,24.57,8.7,6705,4.145,1.608


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [40]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
#Then - convert Outcome to binary array
# Step 1: Label-encode data set
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

encoded_y_train[:5]

array([0, 1, 0, 1, 0])

In [155]:
list(encoded_y_train[:5])

[0, 1, 0, 1, 0]

In [157]:
label_encoder.inverse_transform(list(encoded_y_train[:5]))

array(['CANDIDATE', 'CONFIRMED', 'CANDIDATE', 'CONFIRMED', 'CANDIDATE'],
      dtype=object)

In [42]:
print('Original Class: ' + str(y_train[:5]))

Original Class: 3563    CANDIDATE
4099    CONFIRMED
5460    CANDIDATE
1091    CONFIRMED
5999    CANDIDATE
Name: koi_disposition, dtype: object


In [43]:
# Step 2: One-hot encoding - converts to a binary aray
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical[:5]

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

# Create a Decision Tree Model

In [46]:
from sklearn import tree
classifier = tree.DecisionTreeClassifier()
classifier

DecisionTreeClassifier()

In [94]:
classifier.classes_

[array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32)]

# Train the Model



In [48]:
# Fit model
classifier = classifier.fit(X_train_scaled, y_train_categorical)

In [49]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 1.0
Testing Data Score: 0.6681922196796338


In [93]:
classifier.classes_

[array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32)]

## Prediction

In [143]:
predictions_tree = classifier.predict(X_test_scaled)
predictions_tree[:10]

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [147]:
x = np.argmax(predictions_tree, axis=1)
x

array([0, 2, 2, ..., 0, 2, 2])

In [150]:
prediction_labels = label_encoder.inverse_transform(x)
x

array([0, 2, 2, ..., 0, 2, 2])

In [141]:
predictions_tree = classifier.predict_classes(X_test_scaled)

AttributeError: 'DecisionTreeClassifier' object has no attribute 'predict_classes'

In [140]:
#Prediction
predictions_tree = classifier.predict(X_test_scaled)
predictions_labels = label_encoder.inverse_transform(predictions_tree)

print("Tree Predicted output:")
print(predictions_tree[:5])
print("-" * 20)

print("Tree Predicted lables:")
print(predictions_labels[:5])
print("-" * 20)

print("Tree True output:")
print(y_test[:5])



ValueError: y should be a 1d array, got an array of shape (1748, 3) instead.

In [136]:
predictions_test = label_encoder.inverse_transform(encoded_y_test)
predictions_test

array(['CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', ...,
       'FALSE POSITIVE', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [120]:
for i in range(0,10):
     print(f"Decision Tree: Predicted output: {predictions_tree[i]}, Actual: {y_test[i]}")
#print(predictions_tree[0])

TypeError: 'Series' object is not callable

# Create a Random Forest Model

In [51]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators=200)
randomforest

RandomForestClassifier(n_estimators=200)

# Train the Model

In [52]:
# Fit model
randomforest = randomforest.fit(X_train_scaled, y_train_categorical)

In [53]:
print(f"Training Data Score: {randomforest.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {randomforest.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 1.0
Testing Data Score: 0.7133867276887872


## prediction

In [89]:
#Prediction
predictions_forest = randomforest.predict(X_test_scaled)
print(f"Forest True output: {y_test[:5]}")
print(f"Forest Predicted output: {predictions_forest[:5]}")

Forest True output: 1213         CONFIRMED
5220    FALSE POSITIVE
6688    FALSE POSITIVE
2031         CONFIRMED
5824    FALSE POSITIVE
Name: koi_disposition, dtype: object
Forest Predicted output: [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [55]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
randomforest.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [56]:
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
#Random Hyperparameter Grid
#Create a parameter grid to sample from during fitting

from sklearn.model_selection import RandomizedSearchCV

In [57]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [64]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [65]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pp.pprint(random_grid)

{   'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [67]:
#Random Search Training
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

In [68]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [69]:
# Fit the random search model
rf_random.fit(X_train_scaled, y_train_categorical)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 39.6min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [70]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 70,
 'bootstrap': False}

In [71]:
#Determine if best random search model yielded a better model than base model
#def evaluate(model, X_test_scaled, y_test_categorical):
#    predictions = model.predict(X_test_scaled)
#    errors = abs(predictions - y_test_categorical)
#    mape = 100 * np.mean(errors / y_test_categorical)
#    accuracy = 100 - mape
#    print('Model Performance')
#    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
#    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
#    return accuracy

In [72]:
#base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
#base_model.fit(X_train_scaled, y_train_categorical)
#base_accuracy = evaluate(base_model, X_test_scaled, y_test_categorical)

Model Performance
Average Error: 0.1659 degrees.
Accuracy = nan%.


/Users/J/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
/Users/J/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [74]:
best_random = rf_random.best_estimator_
#random_accuracy = evaluate(best_random, test_features, test_labels)

#print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))


In [75]:
print(f"Training Data Score: {best_random.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {best_random.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 0.9973297730307076
Testing Data Score: 0.7191075514874142


In [73]:
print(f"Training Data Score: {rf_random.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {rf_random.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 0.9973297730307076
Testing Data Score: 0.7191075514874142


In [76]:
# Train the model with GridSearch
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [300, 600, 900, 1200]
}

In [77]:
# Create a based model
rfgrid = RandomForestClassifier()

In [78]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rfgrid, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [79]:
# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train_categorical)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 32.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 59.3min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 78.0min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [3, 5, 7],
                         'n_estimators': [300, 600, 900, 1200]},
             verbose=2)

In [80]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 600}

In [81]:
best_grid = grid_search.best_estimator_


In [82]:
print(f"Training Data Score: {best_grid.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {best_grid.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 0.9967575815372878
Testing Data Score: 0.7196796338672768


In [83]:
print(grid_search.best_score_)

0.6997900290658454


## Predictions

In [90]:
#Prediction
predictions_bestgrid = best_grid.predict(X_test_scaled)
print(f"Forest True output: {y_test[:5]}")
print(f"Forest Predicted output: {predictions_bestgrid[:5]}")

Forest True output: 1213         CONFIRMED
5220    FALSE POSITIVE
6688    FALSE POSITIVE
2031         CONFIRMED
5824    FALSE POSITIVE
Name: koi_disposition, dtype: object
Forest Predicted output: [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Comparison

In [ ]:
#Decision Tree - Model name: classifier
pd.DataFrame({"Predicted": predictions, "Actual": y, "Error": predictions - y})[["Predicted", "Actual", "Error"]]

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)